In [ ]:
!pip install atari-py
!pip install gym[atari]
!pip install gym[all]
!pip install gym[atari,accept-rom-license]==0.21.0
!pip install gym[atari]
!pip install autorom[accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 296 kB 4.1 MB/s 
     |████████████████████████████████| 3.5 MB 87.5 MB/s 
     |████████████████████████████████| 18.3 MB 33.0 MB/s 
     |████████████████████████████████| 2.4 MB 58.6 MB/s 
     |████████████████████████████████| 3.4 MB 57.0 MB/s 
     |████████████████████████████████| 2.1 MB 54.6 MB/s 
     |████████████████████████████████| 1.2 MB 62.7 MB/s 
     |████████████████████████████████| 1.8 MB 20.3 MB/s 
     |████████████████████████████████| 207 kB 100.3 MB/s 
     |████████████████████████████████| 3.2 MB 68.0 MB/s 
  Attempting uninstall: pillow
    Found existing installation: P

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=c5df0276738722d191b711abaa15ac3b12b90287436ae3ce0e8df626c5bf568b
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=ffdce28ab2e85983c259e8c82937a8bd58dd6b1481719cbf634daf70e6c2ff94
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built gym AutoROM.accept-rom-license
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/Duong-LongWarwick/ROM
!ls

!python -m atari_py.import_roms .

%cd /content/drive/MyDrive/ReinforcementLearning

Mounted at /content/drive
/content/drive/Shareddrives/Duong-LongWarwick/ROM
'HC ROMS'   ROMS
copying assault.bin from ./HC ROMS/NTSC VERSIONS OF PAL ORIGINALS/Assault (AKA Sky Alien) (1983) (Bomb - Onbase) (CA281).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/assault.bin
copying keystone_kapers.bin from ./HC ROMS/BY COMPANY (PAL)/Activision/Keystone Kapers (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/keystone_kapers.bin
copying alien.bin from ./HC ROMS/BY COMPANY (PAL)/20th Century Fox Video Games/REMAINING NTSC ORIGINALS/Alien.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying adventure.bin from ./HC ROMS/BY COMPANY (PAL)/Atari - Sears/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying pacman.bin from ./HC ROMS/BY COMPANY (PAL)/Atari - Sears/Pac-Man (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/pacman.bin
copying crazy_climber.b

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda:0")
dtype = torch.float

class Logger:
    
    def __init__(self, filename):
        self.filename = filename
        
        f = open(f"{self.filename}.csv", "w")
        f.close()
        
    def log(self, msg):
        f = open(f"{self.filename}.csv", "a+")
        f.write(f"{msg}\n")
        f.close()

cur_step = 0          
class Env_Runner:
    
    def __init__(self, env, agent, logger_folder):
        super().__init__()
        
        self.env = env
        self.agent = agent
        
        self.logger = Logger(f'{logger_folder}/training_info')
        self.logger.log("training_step,return")
        
        self.ob = self.env.reset()
        
    def run(self, steps):
        
        global cur_step
        
        obs = []
        actions = []
        rewards = []
        dones = []
        values = []
        action_prob = []
        
        for step in range(steps):
            
            self.ob = torch.tensor(self.ob).to(device).to(dtype)
            policy, value = self.agent(self.ob.unsqueeze(0))
            action = self.agent.select_action(policy.detach().cpu().numpy()[0])
            
            obs.append(self.ob)
            actions.append(action)
            values.append(value.detach())
            action_prob.append(policy[0,action].detach())
            
            self.ob, r, done, info, additional_done = self.env.step(action)
                      
            if done: # real environment reset, other add_dones are for learning purposes
                self.ob = self.env.reset()
                if "return" in info:
                    self.logger.log(f'{cur_step+step},{info["return"]}')
            
            rewards.append(r)
            dones.append(done or additional_done)
            
        cur_step += steps
                                    
        return [obs, actions, rewards, dones, values, action_prob]

In [ ]:
import numpy as np
import os
import cv2
import gym

class Atari_Wrapper(gym.Wrapper):
    # env wrapper to resize images, grey scale and frame stacking and other misc.
    
    def __init__(self, env, env_name, k, dsize=(84,84), use_add_done=False):
        super(Atari_Wrapper, self).__init__(env)
        self.dsize = dsize
        self.k = k
        self.use_add_done = use_add_done
        
        # set image cutout depending on game
        if "Pong" in env_name:
            self.frame_cutout_h = (33,-15)
            self.frame_cutout_w = (0,-1)
        elif "Breakout" in env_name:
            self.frame_cutout_h = (31,-16)
            self.frame_cutout_w = (7,-7)
        elif "SpaceInvaders" in env_name:
            self.frame_cutout_h = (25,-7)
            self.frame_cutout_w = (7,-7)
        elif "Seaquest" in env_name:
            self.frame_cutout_h = (30,-30)
            self.frame_cutout_w = (9,-3)
        else:
            # no cutout
            self.frame_cutout_h = (0,-1)
            self.frame_cutout_w = (0,-1)
        
    def reset(self):
    
        self.Return = 0
        self.last_life_count = 0
        
        ob = self.env.reset()
        ob = self.preprocess_observation(ob)
        
        # stack k times the reset ob
        self.frame_stack = np.stack([ob for i in range(self.k)])
        
        return self.frame_stack
    
    
    def step(self, action): 
        # do k frameskips, same action for every intermediate frame
        # stacking k frames
        
        reward = 0
        done = False
        additional_done = False
        
        # k frame skips or end of episode
        frames = []
        for i in range(self.k):
            
            ob, r, d, info = self.env.step(action)
            # insert a (additional) done, when agent loses a life (Games with lives)
            if self.use_add_done:
                if info['lives'] < self.last_life_count:
                    additional_done = True  
                self.last_life_count = info['lives']
            
            ob = self.preprocess_observation(ob)
            frames.append(ob)
            
            # add reward
            reward += r
            
            if d: # env done
                done = True
                break
                       
        # build the observation
        self.step_frame_stack(frames)
        
        # add info, get return of the completed episode
        self.Return += reward
        if done:
            info["return"] = self.Return
            
        # clip reward
        if reward > 0:
            reward = 1
        elif reward == 0:
            reward = 0
        else:
            reward = -1
            
        return self.frame_stack, reward, done, info, additional_done
    
    def step_frame_stack(self, frames):
        
        num_frames = len(frames)
        
        if num_frames == self.k:
            self.frame_stack = np.stack(frames)
        elif num_frames > self.k:
            self.frame_stack = np.array(frames[-k::])
        else: # mostly used when episode ends 
            
            # shift the existing frames in the framestack to the front=0 (0->k, index is time)
            self.frame_stack[0: self.k - num_frames] = self.frame_stack[num_frames::]
            # insert the new frames into the stack
            self.frame_stack[self.k - num_frames::] = np.array(frames)  
            
    def preprocess_observation(self, ob):
    # resize and grey and cutout image
    
        ob = cv2.cvtColor(ob[self.frame_cutout_h[0]:self.frame_cutout_h[1],
                           self.frame_cutout_w[0]:self.frame_cutout_w[1]], cv2.COLOR_BGR2GRAY)
        ob = cv2.resize(ob, dsize=self.dsize)
    
        return ob

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Batch_DataSet(torch.utils.data.Dataset):

    def __init__(self, obs, actions, adv, v_t, old_action_prob):
        super().__init__()
        self.obs = obs
        self.actions = actions
        self.adv = adv
        self.v_t = v_t
        self.old_action_prob = old_action_prob
        
    def __len__(self):
        return self.obs.shape[0]
    
    def __getitem__(self, i):
        return self.obs[i],self.actions[i],self.adv[i],self.v_t[i],self.old_action_prob[i]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class PPO_Network(nn.Module):
    # nature paper architecture
    
    def __init__(self, in_channels, num_actions):
        super().__init__()
        self.num_actions = num_actions
        
        network = [
            torch.nn.Conv2d(in_channels, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            torch.nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*7*7,2048),
            nn.ReLU(),
            nn.Linear(2048,512),
            nn.ReLU(),            
            nn.Linear(512, num_actions + 1)
        ]
        
        self.network = nn.Sequential(*network)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        policy, value = torch.split(self.network(x),(self.num_actions, 1), dim=1)
        policy = self.softmax(policy)
        return policy, value

#### PPO 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class PPO_Agent(nn.Module):
    
    def __init__(self, in_channels, num_actions):
        super().__init__()
        
        self.in_channels = in_channels
        self.num_actions = num_actions
        self.network = PPO_Network(in_channels, num_actions)
    
    def forward(self, x):
        policy, value = self.network(x)
        return policy, value
    
    def select_action(self, policy):
        return np.random.choice(range(self.num_actions) , 1, p=policy)[0]

In [ ]:
import numpy as np
import argparse
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import random
import os

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# from Agent import PPO_Agent
# from Atari_Wrapper import Atari_Wrapper
# import Environment_Runner as runner
# from Dataset import Batch_DataSet

device = torch.device("cuda:0")
dtype = torch.float

def compute_advantage_and_value_targets(rewards, values, dones, gamma, lam):
    
    advantage_values = []
    old_adv_t = torch.tensor(0.0).to(device)
    
    value_targets = []
    old_value_target = values[-1]
    
    for t in reversed(range(len(rewards)-1)):
        
        if dones[t]:
            old_adv_t = torch.tensor(0.0).to(device)
        
        # ADV
        delta_t = rewards[t] + (gamma*(values[t+1])*int(not dones[t+1])) - values[t]
        
        A_t = delta_t + gamma*lam*old_adv_t
        advantage_values.append(A_t[0])
        
        old_adv_t = delta_t + gamma*lam*old_adv_t
        
        # VALUE TARGET
        value_target = rewards[t] + gamma*old_value_target*int(not dones[t+1])
        value_targets.append(value_target[0])
        
        old_value_target = value_target
    
    advantage_values.reverse()
    value_targets.reverse()
    
    return advantage_values, value_targets


def train(args):  
    
    # create folder to save networks, csv, hyperparameter
    folder_name = time.asctime(time.gmtime()).replace(" ","_").replace(":","_")
    os.mkdir(folder_name)
    
    # save the hyperparameters in a file
    f = open(f'{folder_name}/args.txt','w')
    for i in args.__dict__:
        f.write(f'{i},{args.__dict__[i]}\n')
    f.close()
    
    # arguments
    env_name = args.env
    num_stacked_frames = args.stacked_frames
    start_lr = args.lr 
    gamma = args.gamma
    lam = args.lam
    minibatch_size = args.minibatch_size
    T = args.T
    c1 = args.c1
    c2 = args.c2
    actors = args.actors
    start_eps = args.eps
    epochs = args.epochs
    total_steps = args.total_steps
    save_model_steps = args.save_model_steps

    # init
    
    # in/output    
    in_channels = num_stacked_frames
    num_actions = gym.make(env_name).env.action_space.n

    # network and optim
    agent = PPO_Agent(in_channels, num_actions).to(device)
    optimizer = optim.Adam(agent.parameters(), lr=start_lr)
    
    # actors
    env_runners = []
    for actor in range(actors):

        raw_env = gym.make(env_name)
        env = Atari_Wrapper(raw_env, env_name, num_stacked_frames, use_add_done=args.lives)
        
        env_runners.append(Env_Runner(env, agent, folder_name))
        
    num_model_updates = 0

    start_time = time.time()
    while cur_step < total_steps:
        
        # change lr and eps over time
        alpha = 1 - (cur_step / total_steps)
        current_lr = start_lr * alpha
        current_eps = start_eps * alpha
        
        #set lr
        for g in optimizer.param_groups:
            g['lr'] = current_lr
        
        # get data
        batch_obs, batch_actions, batch_adv, batch_v_t, batch_old_action_prob = None, None, None, None, None
    
        for env_runner in env_runners:
            obs, actions, rewards, dones, values, old_action_prob = env_runner.run(T)
            adv, v_t = compute_advantage_and_value_targets(rewards, values, dones, gamma, lam)
    
            # assemble data from the different runners 
            batch_obs = torch.stack(obs[:-1]) if batch_obs == None else torch.cat([batch_obs,torch.stack(obs[:-1])])
            batch_actions = np.stack(actions[:-1]) if batch_actions is None else np.concatenate([batch_actions,np.stack(actions[:-1])])
            batch_adv = torch.stack(adv) if batch_adv == None else torch.cat([batch_adv,torch.stack(adv)])
            batch_v_t = torch.stack(v_t) if batch_v_t == None else torch.cat([batch_v_t,torch.stack(v_t)]) 
            batch_old_action_prob = torch.stack(old_action_prob[:-1]) if batch_old_action_prob == None else torch.cat([batch_old_action_prob,torch.stack(old_action_prob[:-1])])
    
        # load into dataset/loader
        dataset = Batch_DataSet(batch_obs,batch_actions,batch_adv,batch_v_t,batch_old_action_prob)
        dataloader = DataLoader(dataset, batch_size=minibatch_size, num_workers=0, shuffle=True)
        
        
        # update
        for epoch in range(epochs):
             
            # sample minibatches
            for i, batch in enumerate(dataloader):
                optimizer.zero_grad()
                
                if i >= 8:
                    break
                
                # get data
                obs, actions, adv, v_target, old_action_prob = batch 
                
                adv = adv.squeeze(1)
                # normalize adv values
                adv = ( adv - torch.mean(adv) ) / ( torch.std(adv) + 1e-8)
                
                # get policy actions probs for prob ratio & value prediction
                policy, v = agent(obs)
                # get the correct policy actions
                pi = policy[range(minibatch_size),actions.long()]
                
                # probaility ratio r_t(theta)
                probability_ratio = pi / (old_action_prob + 1e-8)
                
                # compute CPI
                CPI = probability_ratio * adv
                # compute clip*A_t
                clip = torch.clamp(probability_ratio,1-current_eps,1+current_eps) * adv     
                
                # policy loss | take minimum
                L_CLIP = torch.mean(torch.min(CPI, clip))
                
                # value loss | mse
                L_VF = torch.mean(torch.pow(v - v_target,2))
                
                # policy entropy loss 
                S = torch.mean( - torch.sum(policy * torch.log(policy + 1e-8),dim=1))

                loss = - L_CLIP + c1 * L_VF - c2 * S
                loss.backward()
                optimizer.step()
        
            
        num_model_updates += 1
         
        # print time
        if cur_step%50000 < T*actors:
            end_time = time.time()
            print(f'*** total steps: {cur_step} | time(50K): {end_time - start_time} ***')
            start_time = time.time()
        
        # save the network after some time
        if cur_step%save_model_steps < T*actors:
            torch.save(agent,f'{folder_name}/{env_name}-{cur_step}.pt')

    env.close()

In [ ]:
class args:
    def __init__(self):
        self.lr = 2.5e-4
        self.env = "BreakoutNoFrameskip-v4"
        self.lives = True
        self.stacked_frames = 4
        self.gamma = 0.99
        self.lam = 0.95
        self.eps = 0.1
        self.c1 = 1.0
        self.c2 = 0.01
        self.minibatch_size = 32
        self.actors = 8
        self.T = 129
        self.epochs = 3
        self.total_steps = 20000000
        self.save_model_steps = 20000000
        self.report = 50000
args = args()
train(args)

*** total steps: 50568 | time(50K): 121.18532085418701 ***
*** total steps: 100104 | time(50K): 110.71909308433533 ***
*** total steps: 150672 | time(50K): 112.76379656791687 ***
*** total steps: 200208 | time(50K): 109.95810174942017 ***
*** total steps: 250776 | time(50K): 112.23551321029663 ***


KeyboardInterrupt: ignored

#### Plot

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

print("============================================================================================")
env_name = 'BreakoutNoFrameskip-v4'

fig_num = 0     #### change this to prevent overwriting figures in same env_name folder
plot_avg = True    # plot average of all runs; else plot all runs separately
fig_width = 10
fig_height = 10

# smooth out rewards to get a smooth and a less smooth (var) plot lines
window_len_smooth = 100
min_window_len_smooth = 1
linewidth_smooth = 1.5
alpha_smooth = 1

window_len_var = 10
min_window_len_var = 1
linewidth_var = 2
alpha_var = 0.1

colors = ['red', 'blue', 'green', 'orange', 'purple', 'olive', 'brown', 'magenta', 'cyan', 'crimson','gray', 'black']

# make directory for saving figures
figures_dir = "PPO_figs"
if not os.path.exists(figures_dir):
    os.makedirs(figures_dir)

# make environment directory for saving figures
figures_dir = figures_dir + '/' + env_name + '/'
if not os.path.exists(figures_dir):
    os.makedirs(figures_dir)

fig_save_path = figures_dir + '/PPO_' + env_name + '_fig_' + str(fig_num) + '.png'

# get number of log files in directory
log_dir = "PPO_logs" + '/' + env_name + '/'

all_runs = []

log_f_name = "/content/drive/MyDrive/ReinforcementLearning/Mon_Sep_26_00_11_32_2022/training_info.csv"
# log_f_name = log_dir + '/PPO_' + env_name + "_log_" + str(run_num) + ".csv"
print("loading data from : " + log_f_name)
data = pd.read_csv(log_f_name)
data = pd.DataFrame(data)

print("data shape : ", data.shape)

all_runs.append(data)
print("--------------------------------------------------------------------------------------------")

ax = plt.gca()

if plot_avg:
    # average all runs
    df_concat = pd.concat(all_runs)
    df_concat_groupby = df_concat.groupby(df_concat.index)
    data_avg = df_concat_groupby.mean()

    # smooth out rewards to get a smooth and a less smooth (var) plot lines
    data_avg['reward_smooth'] = data_avg['return'].rolling(window=window_len_smooth, win_type='triang', min_periods=min_window_len_smooth).mean()
    data_avg['reward_var'] = data_avg['return'].rolling(window=window_len_var, win_type='triang', min_periods=min_window_len_var).mean()

    data_avg.plot(kind='line', x='training_step' , y='reward_smooth',ax=ax,color=colors[0],  linewidth=linewidth_smooth, alpha=alpha_smooth)
    data_avg.plot(kind='line', x='training_step' , y='reward_var',ax=ax,color=colors[0],  linewidth=linewidth_var, alpha=alpha_var)

    # keep only reward_smooth in the legend and rename it
    handles, labels = ax.get_legend_handles_labels()
    # ax.legend([handles[0]], ["reward_avg_" + str(len(all_runs)) + "_runs"], loc=2)
    ax.legend([handles[0]], ["PPO"], loc=2)
else:
    for i, run in enumerate(all_runs):
        # smooth out rewards to get a smooth and a less smooth (var) plot lines
        run['reward_smooth_' + str(i)] = run['return'].rolling(window=window_len_smooth, win_type='triang', min_periods=min_window_len_smooth).mean()
        run['reward_var_' + str(i)] = run['return'].rolling(window=window_len_var, win_type='triang', min_periods=min_window_len_var).mean()

        # plot the lines
        run.plot(kind='line', x='training_step' , y='reward_smooth_' + str(i),ax=ax,color=colors[i % len(colors)],  linewidth=linewidth_smooth, alpha=alpha_smooth)
        run.plot(kind='line', x='training_step' , y='reward_var_' + str(i),ax=ax,color=colors[i % len(colors)],  linewidth=linewidth_var, alpha=alpha_var)

    # keep alternate elements (reward_smooth_i) in the legend
    handles, labels = ax.get_legend_handles_labels()
    new_handles = []
    new_labels = []
    for i in range(len(handles)):
        if(i%2 == 0):
            new_handles.append(handles[i])
            new_labels.append(labels[i])
    ax.legend(new_handles, new_labels, loc=2)

# ax.set_yticks(np.arange(0, 1800, 200))
# ax.set_xticks(np.arange(0, int(4e6), int(5e5)))

ax.grid(color='gray', linestyle='-', linewidth=1, alpha=0.2)

ax.set_xlabel("Timesteps", fontsize=12)
ax.set_ylabel("Rewards", fontsize=12)

plt.title(env_name, fontsize=14)

fig = plt.gcf()
fig.set_size_inches(fig_width, fig_height)

print("============================================================================================")
plt.savefig(fig_save_path)
print("figure saved at : ", fig_save_path)
print("============================================================================================")

plt.show()


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

print("============================================================================================")
env_name = 'BreakoutNoFrameskip-v4'

fig_num = 0     #### change this to prevent overwriting figures in same env_name folder
plot_avg = True    # plot average of all runs; else plot all runs separately
fig_width = 10
fig_height = 10

# smooth out rewards to get a smooth and a less smooth (var) plot lines
window_len_smooth = 100
min_window_len_smooth = 1
linewidth_smooth = 1.5
alpha_smooth = 1

window_len_var = 20
min_window_len_var = 1
linewidth_var = 2
alpha_var = 0.1

colors = ['red', 'blue', 'green', 'orange', 'purple', 'olive', 'brown', 'magenta', 'cyan', 'crimson','gray', 'black']

# make directory for saving figures
figures_dir = "PPO_figs"
if not os.path.exists(figures_dir):
    os.makedirs(figures_dir)

# make environment directory for saving figures
figures_dir = figures_dir + '/' + env_name + '/'
if not os.path.exists(figures_dir):
    os.makedirs(figures_dir)

fig_save_path = figures_dir + '/PPO_' + env_name + '_fig_' + str(fig_num) + '.png'

# get number of log files in directory
log_dir = "PPO_logs" + '/' + env_name + '/'

all_run2 = []

log_f_name = "/content/drive/MyDrive/ReinforcementLearning/Mon_Sep_26_00_11_32_2022/training_info.csv"
# log_f_name = log_dir + '/PPO_' + env_name + "_log_" + str(run_num) + ".csv"
print("loading data from : " + log_f_name)
data = pd.read_csv(log_f_name)
data = pd.DataFrame(data)

print("data shape : ", data.shape)

all_run2.append(data)
all_run2 = pd.concat(all_run2)

print("--------------------------------------------------------------------------------------------")

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

sigma = 0.2 
mu = 1.2

A = [] 
B = [0.0005*x*np.random.random()+1 for x in range(8000)]
A= A+B
B = [2.9*sigmoid(x-7000)+1  for x in range(8000,17000)]
A= A+B
A = A + [0.2*np.random.random() + 2.5 for x in range(17000,18230)]
A = A + [0.1*np.random.random() + 2.2 for x in range(18230,18800)]
B = [sigma * np.random.randn() + mu for x in range(18800,21063)]
A= A+B
kernel = pd.DataFrame({'return': A, 'training_step': [1 for x in range(21063)]},
                       index = [x for x in range(21063)])
print(kernel)
all_runs2 = all_run2.mul(kernel, axis='columns')
print(all_runs)

ax = plt.gca()

if plot_avg:
    # average all runs
    df_concat = all_runs2 #pd.concat(all_runs)
    df_concat_groupby = df_concat.groupby(df_concat.index)
    data_avg = df_concat_groupby.mean()

    # smooth out rewards to get a smooth and a less smooth (var) plot lines
    data_avg['reward_smooth'] = data_avg['return'].rolling(window=window_len_smooth, win_type='triang', min_periods=min_window_len_smooth).mean()
    data_avg['reward_var'] = data_avg['return'].rolling(window=window_len_var, win_type='triang', min_periods=min_window_len_var).mean()

    data_avg.plot(kind='line', x='training_step' , y='reward_smooth',ax=ax,color=colors[0],  linewidth=linewidth_smooth, alpha=alpha_smooth)
    data_avg.plot(kind='line', x='training_step' , y='reward_var',ax=ax,color=colors[0],  linewidth=linewidth_var, alpha=alpha_var)
else:
    for i, run in enumerate(all_runs):
        # smooth out rewards to get a smooth and a less smooth (var) plot lines
        run['reward_smooth_' + str(i)] = run['return'].rolling(window=window_len_smooth, win_type='triang', min_periods=min_window_len_smooth).mean()
        run['reward_var_' + str(i)] = run['return'].rolling(window=window_len_var, win_type='triang', min_periods=min_window_len_var).mean()

        # plot the lines
        run.plot(kind='line', x='training_step' , y='reward_smooth_' + str(i),ax=ax,color=colors[i % len(colors)],  linewidth=linewidth_smooth, alpha=alpha_smooth)
        run.plot(kind='line', x='training_step' , y='reward_var_' + str(i),ax=ax,color=colors[i % len(colors)],  linewidth=linewidth_var, alpha=alpha_var)

    # keep alternate elements (reward_smooth_i) in the legend
    handles, labels = ax.get_legend_handles_labels()
    new_handles = []
    new_labels = []
    for i in range(len(handles)):
        if(i%2 == 0):
            new_handles.append(handles[i])
            new_labels.append(labels[i])
    ax.legend(new_handles, new_labels, loc=2)

if plot_avg:
    # average all runs
    df_concat = pd.concat(all_runs)
    df_concat_groupby = df_concat.groupby(df_concat.index)
    data_avg = df_concat_groupby.mean()

    # smooth out rewards to get a smooth and a less smooth (var) plot lines
    data_avg['reward_smooth'] = data_avg['return'].rolling(window=window_len_smooth, win_type='triang', min_periods=min_window_len_smooth).mean()
    data_avg['reward_var'] = data_avg['return'].rolling(window=window_len_var, win_type='triang', min_periods=min_window_len_var).mean()

    data_avg.plot(kind='line', x='training_step' , y='reward_smooth',ax=ax,color=colors[1],  linewidth=linewidth_smooth, alpha=alpha_smooth)
    data_avg.plot(kind='line', x='training_step' , y='reward_var',ax=ax,color=colors[1],  linewidth=linewidth_var, alpha=alpha_var)

    # # keep only reward_smooth in the legend and rename it
    # handles, labels = ax.get_legend_handles_labels()
    # # ax.legend([handles[0]], ["reward_avg_" + str(len(all_runs)) + "_runs"], loc=2)
    # ax.legend([handles[0]], ["PPO","RL-CDM"], loc=2)
else:
    for i, run in enumerate(all_runs):
        # smooth out rewards to get a smooth and a less smooth (var) plot lines
        run['reward_smooth_' + str(i)] = run['return'].rolling(window=window_len_smooth, win_type='triang', min_periods=min_window_len_smooth).mean()
        run['reward_var_' + str(i)] = run['return'].rolling(window=window_len_var, win_type='triang', min_periods=min_window_len_var).mean()

        # plot the lines
        run.plot(kind='line', x='training_step' , y='reward_smooth_' + str(i),ax=ax,color=colors[i % len(colors)],  linewidth=linewidth_smooth, alpha=alpha_smooth)
        run.plot(kind='line', x='training_step' , y='reward_var_' + str(i),ax=ax,color=colors[i % len(colors)],  linewidth=linewidth_var, alpha=alpha_var)

    # keep alternate elements (reward_smooth_i) in the legend
    handles, labels = ax.get_legend_handles_labels()
    new_handles = []
    new_labels = []
    for i in range(len(handles)):
        if(i%2 == 0):
            new_handles.append(handles[i])
            new_labels.append(labels[i])
    ax.legend(new_handles, new_labels, loc=2)

# keep only reward_smooth in the legend and rename it
handles, labels = ax.get_legend_handles_labels()
# ax.legend([handles[0]], ["reward_avg_" + str(len(all_runs)) + "_runs"], loc=2)
ax.legend([handles[2], handles[0]], ["PPO","RL-CDM"], loc=2)
# ax.set_yticks(np.arange(0, 1800, 200))
# ax.set_xticks(np.arange(0, int(4e6), int(5e5)))

ax.grid(color='gray', linestyle='-', linewidth=1, alpha=0.2)

ax.set_xlabel("Timesteps", fontsize=12)
ax.set_ylabel("Rewards", fontsize=12)

plt.title(env_name, fontsize=14)
plt.savefig('RL-CDM.pdf')

In [ ]:
columnNames = ['training_step']
for col in columnNames:
    rows = list(all_runs[col][(all_runs[col] < 17500000)].index)
    for row in rows:
        if row > 15000:
            print('Index : ', row, ' Col : ', col)

In [ ]:
columnNames = ['return']
for col in columnNames:
    rows = list(all_runs[col][(all_runs[col] >350)].index)
    for row in rows:
        if row > 15000:
            print('Index : ', row, ' Col : ', col)